In [21]:
import random as rd
import time as tm
import os

In [22]:
# ---------- 클래스 정의 ----------
class Bomb_game:
    '''2인용 폭탄게임을 하기 위한 여러가지 기능(메소드)을 제공하는 클래스이다.'''
    Exis_card = {'폭탄': 1, '제거': 2, '셔플': 3, '투시': 3, '강탈': 3,
                 '스킵': 3, '밑장빼기': 3, '프라이팬x1': 4, '프라이팬x3': 2} # 카드이름 : 카드개수

    How_to_play = '''
    먼저 선을 정하고, 선부터 돌아가면서 턴을 진행한다.
    자기 턴에는 카드덱에서 카드를 뽑거나, 자신이 들고 있는 카드를 사용할 수 있다.
    카드덱에서 카드를 뽑으면 1턴이 소모되고, 카드를 사용하는 것은 턴을 소모하지 않는다.(특정 카드 제외)''' # 게임방법

    def __init__(self, name):
        self.name = name      # 이름
        self.turn = False     # 자기 턴인지
        self.num = 1          # 턴의 수
        self.frynum = 0       # 프라이팬 턴의 수
        self.hav_card = []    # 가지고 있는 카드
        self.alive = True     # 살아있는지

    def __str__(self):
        return self.name

    @staticmethod
    def setting_deck():
        '''
        시작할 때 덱을 섞는 메소드
        단, 다음의 조건을 만족해야한다.
        1. 폭탄은 처음 4장에서 나올 수 없다.
        2. 폭탄보다 위에 최소 1개의 제거 카드가 있어야한다.
        3. 모든 카드는 3장 이상 연속으로 나올 수 없다.

        섞은 결과는 클래스 속성 Deck 에 들어간다.
        '''
        cond1 = True
        cond2 = True
        cond3 = True

        Bomb_game.Deck = []
        for k, v in Bomb_game.Exis_card.items() :
            for n in range(v) :
                Bomb_game.Deck.append(k)

        while cond1 or cond2 or cond3 :
            rd.shuffle(Bomb_game.Deck)
            if Bomb_game.Deck.index('폭탄') < 4 : # 조건 충족 안 되는 경우
                cond1 = True
            else :
                cond1 = False

            if Bomb_game.Deck.index('폭탄') < Bomb_game.Deck.index('제거') : # 조건 충족 안 되는 경우
                cond2 = True
            else :
                cond2 = False

            for i in range(len(Bomb_game.Deck) - 2) :
                if (Bomb_game.Deck[i] == Bomb_game.Deck[i+1]) and (Bomb_game.Deck[i+1] == Bomb_game.Deck[i+2]) : # 조건 충족 안 되는 경우
                    cond3 = True
                    break
                else :
                    cond3 = False

    @staticmethod
    def choose_first(instn_1, instn_2):
        '''
        선을 정하는 메소드
        인스턴스 두 개를 받아서 랜덤으로 선을 정하고, 선으로 정해진 인스턴스를 리턴한다.
        선으로 정해진 인스턴스의 self.turn 을 True 로 바꾼다.
        '''
        first = rd.choice((instn_1, instn_2))
        first.turn = True
        return first

    @staticmethod
    def help_word():
        '''
        도움말을 출력하는 메소드
        '''
        print(f'카드 종류\n\
        제거 : {Bomb_game.remove.__doc__}\n\
        셔플 : {Bomb_game.shuffle.__doc__}\n\
        투시 : {Bomb_game.xray.__doc__}\n\
        강탈 : {Bomb_game.steal.__doc__}\n\
        스킵 : {Bomb_game.skip.__doc__}\n\
        밑장빼기 : {Bomb_game.underdraw.__doc__}\n\
        프라이팬 : {Bomb_game.frypan.__doc__}')
        print(f'게임 방법\n{Bomb_game.How_to_play}\n')

    def show_card(self):
        '''
        현재 들고 있는 카드 목록을 출력하는 메소드
        '''
        print('현재 들고 있는 카드 목록은')
        print(self.hav_card, '입니다')

    def draw_card(self):
        '''
        카드를 뽑는 메소드
        setting_deck() 메소드로 만든 Deck 속성에서 왼쪽에서부터 한 장을 뽑고, 그걸 리턴한다.
        뽑힌 카드는 Deck 에서 제거하고, self.hav_card 에 추가한다. 턴을 1 소모한다.
        '''
        card = Bomb_game.Deck.pop(0)
        self.hav_card.append(card)
        self.num -= 1
        if self.frynum > 0 :
            self.frynum -= 1
        return card

    # 카드를 사용할 때 호출되는 메소드들이다.
    # 메소드가 호출될 때 self.hav_card 에서 사용한 카드를 제거한다.
    def remove(self):
        '''
        폭탄을 뽑았을 때 이 카드를 소모해서 한번 살고, 카드덱의 랜덤한 위치에 폭탄이 들어간다. (남은 턴 개수와 상관없이 턴이 넘어간다.)
        '''
        self.hav_card.remove('제거')
        self.hav_card.remove('폭탄')
        Bomb_game.Deck.insert(rd.randrange(len(Bomb_game.Deck) + 1), '폭탄')
        print('제거 카드를 사용하여 살아남았습니다..!')

    def shuffle(self):
        '''
        카드덱을 랜덤으로 섞는다.
        '''
        self.hav_card.remove('셔플')
        rd.shuffle(Bomb_game.Deck)

    def xray(self):
        '''
        카드덱의 가장 위에 있는 3장을 본다.
        '''
        self.hav_card.remove('투시')
        try:
            print(f'윗장부터 {Bomb_game.Deck[0]}, {Bomb_game.Deck[1]}, {Bomb_game.Deck[2]} 입니다 (5초 후 사라집니다)', end= '')
        except IndexError:
            if len(Bomb_game.Deck) == 2 :
                print(f'윗장부터 {Bomb_game.Deck[0]}, {Bomb_game.Deck[1]} 입니다 (5초 후 사라집니다)', end= '')
            elif len(Bomb_game.Deck) == 1 :
                print(f'윗장부터 {Bomb_game.Deck[0]} 입니다 (5초 후 사라집니다)', end= '')
        tm.sleep(5)
        print('\r ')

    def steal(self, oppo):
        '''
        상대의 카드 하나를 뺏어온다. (뺏어올 카드는 상대가 정한다.)
        '''
        self.hav_card.remove('강탈')
        print(f'상대의 카드 목록은 {oppo.hav_card} 입니다')
        print(f'어떤 카드를 줄 지 {oppo} 님이 선택해주세요')
        give_card = input()
        #os.system('cls')

        while give_card not in oppo.hav_card :
            print('가지고 있는 카드 중에서 선택해주세요')
            print(f'카드 목록은 {oppo.hav_card} 입니다')
            give_card = input()
            #os.system('cls')

        oppo.hav_card.remove(give_card)
        self.hav_card.append(give_card)
        print(f'{oppo} 님에게서 {give_card} 카드를 빼앗았습니다!')

    def skip(self):
        '''
        턴을 하나 넘긴다.
        '''
        self.hav_card.remove('스킵')
        self.num -= 1
        if self.frynum > 0 :
            self.frynum -= 1

    def underdraw(self):
        '''
        카드덱 가장 아래의 있는 카드를 뽑는다. (턴을 소모하지 않는다.)
        '''
        self.hav_card.remove('밑장빼기')
        card = Bomb_game.Deck.pop(-1)
        self.hav_card.append(card)
        return card

    def frypan(self, oppo, n = 1):
        '''
        x1과 x3 두 종류가 있다.상대 또는 자신에게 사용할 수 있으며, 지목받은 자에게 1 또는 3턴을 부여한다.
        자신이 프라이팬 턴을 가지고 있는 상황에서 사용하면 턴이 추가된다.
        '''
        if n == 1 :
            self.hav_card.remove('프라이팬x1')
        elif n == 3 :
            self.hav_card.remove('프라이팬x3')
        print('누구에게 사용할 지 선택해주세요')
        print("나에게 사용하려면 '0', 상대에게 사용하려면 '1' 을 입력하세요")
        fry_who = input()
        #os.system('cls')

        while (fry_who != '0') and (fry_who != '1') :
            print("나에게 사용하려면 '0', 상대에게 사용하려면 '1' 을 입력하세요")
            fry_who = input()
            #os.system('cls')

        if (not self.frynum) and (not int(fry_who)) : # 프라이팬 턴이 0일 때 / 나에게 사용
            self.frynum = n
            self.num = self.frynum

        elif self.frynum and (not int(fry_who)) : # 프라이팬 턴이 0이 아닐 때 / 나에게 사용
            self.frynum += n
            self.num = self.frynum

        elif (not self.frynum) and int(fry_who) : # 프라이팬 턴이 0일 때 / 상대에게 사용
            oppo.frynum = n
            oppo.num = oppo.frynum
            self.num = 0

        elif self.frynum and int(fry_who) : # 프라이팬 턴이 0이 아닐 때 / 상대에게 사용
            oppo.frynum = self.frynum + n
            oppo.num = oppo.frynum
            self.num = 0


    # 폭탄 카드를 뽑고, 제거 카드가 있을 때 실행되는 메소드
    def bomb(self):
        '''
        폭탄 카드를 뽑고, 제거카드로 제거하지 못할 시 폭탄이 터지고 게임에서 패배한다.
        '''
        print('폭탄이 터집니다!!')
        for i in range(3, 0, -1) :
            print(i)
            tm.sleep(1)
        print('BOOM!!')
        self.alive = False

In [23]:
# ---------- 예외 함수 ----------
def two_name(names):
    if len(names) != 2 :
        raise ValueError('두 분의 이름을 입력해주세요')

def dif_name(names):
    if names[0].replace(' ', '') == names[1].replace(' ', '') :
        raise ValueError('이름을 다르게 적어주세요')

def no_empty_name(names):
    if (names[0].replace(' ', '') == '') or (names[1].replace(' ', '') == '') :
        raise ValueError('한 글자 이상의 이름을 입력해주세요')

In [24]:
while True :
    # ---------- 시작 세팅 ----------
    print('폭탄게임에 오신 걸 환영합니다!')
    while True :
        try:
            print("폭탄게임을 하려고 하는 두 분의 이름을 ','(쉼표)로 구분하여 적어주세요")
            name_list = input().split(',')
            #os.system('cls')

            two_name(name_list)
            dif_name(name_list)
            no_empty_name(name_list)

        except Exception as e:
            print('Error :', e)
            continue

        print(f'{name_list[0]} 님과 {name_list[1]} 님 맞으신가요?')
        print("맞으시다면 'yes', 아니라면 'no'를 입력해주세요")
        check_name = input()
        #os.system('cls')

        while (check_name != 'yes') and (check_name != 'no') :
            print('yes 또는 no 를 입력해주세요')
            check_name = input()
            #os.system('cls')

        if check_name == 'yes' :
            break
        elif check_name == 'no' :
            continue

    plyer1, plyer2 = Bomb_game(name_list[0]), Bomb_game(name_list[1])
    print('좋습니다')
    print('그럼 게임을 시작하겠습니다')
    tm.sleep(3)
    #os.system('cls')

    print('먼저 선을 정하겠습니다')
    for _ in range(3) :
        tm.sleep(1)
        print('.')

    first = Bomb_game.choose_first(plyer1, plyer2)
    tm.sleep(2)
    print(f'선은 {first} 님 입니다!')
    tm.sleep(3)
    #os.system('cls')


    # ---------- 게임 시작 ----------
    Bomb_game.setting_deck()
    print('게임을 시작합니다')
    while plyer1.alive and plyer2.alive :
        #os.system('cls')
        if plyer1.turn :
            print(f'{plyer1} 님의 턴!')
            now_turn = plyer1
            next_turn = plyer2
        elif plyer2.turn :
            print(f'{plyer2} 님의 턴!')
            now_turn = plyer2
            next_turn = plyer1
        tm.sleep(2)

        if now_turn.frynum == 0 :
            now_turn.num = 1
        while now_turn.num > 0 :
            draw_use = None
            getout = True # 탈출 커맨드
            while draw_use != '0' and getout :
                print(f'\n현재 남은 턴 수는 {now_turn.num} 입니다.  {len(Bomb_game.Deck)} 장 남았습니다')
                print("카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요")
                print("(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)")
                draw_use = input()
                #os.system('cls')

                if draw_use == 'show' :
                    now_turn.show_card()

                elif draw_use == 'help' :
                    Bomb_game.help_word()

                elif draw_use == '1' :
                    use_card = '제거'
                    while use_card == '제거' :
                        now_turn.show_card()
                        print("어떤 카드를 사용할 지 선택하세요 (뒤로 가려면 'back'을 입력하세요)")
                        use_card = input()
                        #os.system('cls')

                        while (use_card != 'back') and (use_card not in now_turn.hav_card) :
                            print(f"s{use_card} 을(를) 찾을 수 없습니다. 다시 입력해주세요 (뒤로 가려면 'back'을 입력하세요)")
                            use_card = input()
                            #os.system('cls')

                        if use_card == 'back' :
                            break
                        elif use_card == '제거' :
                            print('제거 카드는 직접 사용할 수 없습니다. 다른 카드를 선택하세요')
                            continue
                        else :
                            print(f'{use_card} 카드를 사용합니다')
                            if use_card == '셔플' :
                                now_turn.shuffle()

                            elif use_card == '투시' :
                                now_turn.xray()

                            elif use_card == '강탈' :
                                now_turn.steal(next_turn)

                            elif use_card == '스킵' :
                                now_turn.skip()
                                getout = False

                            elif use_card == '밑장빼기' :
                                card = now_turn.underdraw()
                                print(f'{card} 카드를 뽑으셨습니다!')
                                tm.sleep(3)

                            elif use_card == '프라이팬x1' :
                                now_turn.frypan(next_turn, 1)
                                getout = False

                            elif use_card == '프라이팬x3' :
                                now_turn.frypan(next_turn, 3)
                                getout = False

            if draw_use == '0' :
                card = now_turn.draw_card()
                print(f'{card} 카드를 뽑으셨습니다!')
                tm.sleep(3)

                if card == '폭탄' :
                    now_turn.num = 0
                    now_turn.frynum = 0
                    if '제거' in now_turn.hav_card :
                        now_turn.remove()
                        tm.sleep(2)
                        #os.system('cls')

                    else :
                        now_turn.bomb()
                        tm.sleep(2)
                        break

            if now_turn.num == 0 : # 턴을 다 소모했을 경우
                now_turn.turn = False
                next_turn.turn = True

    print()
    print('='*50)
    if plyer1.alive :
        print(f'축하합니다! {plyer1} 님의 승리입니다!!')
    elif plyer2.alive :
        print(f'축하합니다! {plyer2} 님의 승리입니다!!')
    print('='*50)
    print()

    print("다시 하시려면 'yes', 그만하시려면 'no' 를 입력해주세요")
    restart = input()
    #os.system('cls')

    while (restart != 'yes') and (restart != 'no') :
        print("다시 하시려면 'yes', 그만하시려면 'no' 를 입력해주세요")
        restart = input()

    if restart == 'yes' :
        continue
    elif restart == 'no' :
        break

폭탄게임에 오신 걸 환영합니다!
폭탄게임을 하려고 하는 두 분의 이름을 ','(쉼표)로 구분하여 적어주세요
a,b
a 님과 b 님 맞으신가요?
맞으시다면 'yes', 아니라면 'no'를 입력해주세요
yes
좋습니다
그럼 게임을 시작하겠습니다
먼저 선을 정하겠습니다
.
.
.
선은 b 님 입니다!
게임을 시작합니다
b 님의 턴!

현재 남은 턴 수는 1 입니다.  24 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)
0
스킵 카드를 뽑으셨습니다!
a 님의 턴!

현재 남은 턴 수는 1 입니다.  23 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)
0
제거 카드를 뽑으셨습니다!
b 님의 턴!

현재 남은 턴 수는 1 입니다.  22 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)
0
프라이팬x1 카드를 뽑으셨습니다!
a 님의 턴!

현재 남은 턴 수는 1 입니다.  21 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)
0
밑장빼기 카드를 뽑으셨습니다!
b 님의 턴!

현재 남은 턴 수는 1 입니다.  20 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를 보고싶으면 'show', 도움이 필요하다면 'help' 를 입력하세요)
0
강탈 카드를 뽑으셨습니다!
a 님의 턴!

현재 남은 턴 수는 1 입니다.  19 장 남았습니다
카드를 뽑을거면 '0', 카드를 사용할거면 '1' 을 입력하세요
(자신이 들고있는 카드를